## Заданы координаты прямоугольников на плоскости (не более 15). Вывести в отдельных строках номера прямоугольников, образующих группы пересекающихся (каждая группа в одной отдельной строке).

### Работу выполнил: Кирилл Николаев, 15ФПЛ

In [1]:
import sys, random, math

In [2]:
while True:
    try:
        num = int(input('Введите количество прямоугольников, не менее двух, но не более пятнадцати: '))
        if 1 < num <= 15:
            break
        else:
            print('Не соответствует условию.')
    except ValueError:
        print('Невозможное количество прямоугольников.')

Введите количество прямоугольников, не менее двух, но не более пятнадцати: 15


Возьмём 30 условных единиц ширины и длины на прямоугольник: то есть при количестве треугольников 15, поле 450х450, при 8 треугольниках, 240х240, и т.д. При этом пусть размер треугольника будет случайный, длина стороны не более 1/60 от стороны поля (потому что иначе псевдорандом оставляет всего одну группу пересекающихся прямоугольников в итоге, и это непорядок)

In [70]:
field = num * 30
side = round(num * 0.5) #так как рандом дальше требует int
print(field)
print(side)

450
8


Функция для генерации случайного прямоугольника

In [57]:
def randomrectangle(): 
    x10 = random.randrange(field) #координаты точек-середин противположных сторон,
    y10 = random.randrange(field)
    x20 = random.randrange(field)
    y20 = random.randrange(field)
    L = random.randrange(side) #длины которых L нам известны
    if x10 == x20: #alligned по вертикали
        p1 = [x10-0.5*L, y10]
        p2 = [x10+0.5*L, y10]
        p3 = [x20+0.5*L, y20]
        p4 = [x20-0.5*L, y20]
    elif y10 == y20: #alligned по горизонтали
        p1 = [x10, y10+0.5*L]
        p2 = [x20, y10+0.5*L]
        p3 = [x20, y10-0.5*L]
        p4 = [x10, y10-0.5*L]
    else: # allignment'а нет, есть отклонение
        m1 = (y20-y10) / (x20-x10) #отклонение одной стороны
        m2 = -1 / m1 #отклонение другой стороны
        dx = math.sqrt(L**2 / (1 + m2**2)) / 2
        dy = m2*dx
        p1 = [round(x10+dx,2),round(y10+dy,2)]
        p2 = [round(x10-dx,2),round(y10-dy,2)]
        p3 = [round(x20+dx,2),round(y20+dy,2)]
        p4 = [round(x20-dx,2),round(y20-dy,2)]
    corners = [p1,p2,p3,p4]
    return corners

Создаём алгоритм для сортировки точек по часовой стрелке относительно избранной вершины (в противном случае площадь будет определяться как 0):

In [58]:
def makeclockwise(r):
    origin = r[0] # исходная точка
    refvec = [0, 1] # вектор, направленный вверх, для определения угла
    
    def clockwiseangle_and_distance(point):
        # Вектор между точкой и исходной точкой-вершиной: v = p - o
        vector = [point[0]-origin[0], point[1]-origin[1]]
        # длина вектора: ||v||
        lenvector = math.hypot(vector[0], vector[1])
        # если длина - 0, угла нет
        if lenvector == 0:
            return -math.pi, 0
        # нормализуем вектор: v/||v||
        normalized = [vector[0]/lenvector, vector[1]/lenvector]
        dotprod  = normalized[0]*refvec[0] + normalized[1]*refvec[1]     # x1*x2 + y1*y2
        diffprod = refvec[1]*normalized[0] - refvec[0]*normalized[1]     # x1*y2 - y1*x2
        angle = math.atan2(diffprod, dotprod)
        # отрицательные углы - против часовой стрелки; отнимаем от 2*pi (360 градусов)
        if angle < 0:
            return 2*math.pi+angle, lenvector
        # сначала угол, потому что главный критерий,
        # но если угол одинаковый, то с более коротким расстоянием первый.
        return angle, lenvector
    return sorted(r, key=clockwiseangle_and_distance)

Создаём словарь, нумеруя прямоугольники и сортируя точки по часовой стрелке (в противном случае при подстановке вершин в неверном порядке площадь будет получаться равной нулю)

In [66]:
count = 1
rectdict = {} #Словарь сгенерированных случайных прямоугольников
while count <= num:
    rectdict[count]=makeclockwise(randomrectangle())
    count += 1
print(rectdict)

{1: [[223.48, 361.34], [374.48, 194.34], [371.52, 191.66], [220.52, 358.66]], 2: [[78.09, 373.64], [176.09, 188.64], [169.91, 185.36], [71.91, 370.36]], 3: [[199.47, 101.28], [196.53, 100.72], [172.53, 227.72], [175.47, 228.28]], 4: [[22.0, 277.0], [22.0, 277.0], [196.0, 262.0], [196.0, 262.0]], 5: [[223.74, 90.78], [339.26, 365.22], [344.74, 362.78], [218.26, 93.22]], 6: [[441.01, 14.5], [42.01, 12.5], [41.99, 17.5], [440.99, 19.5]], 7: [[32.62, 349.9], [96.38, 391.1], [99.62, 384.9], [29.38, 356.1]], 8: [[101.7, 153.31], [96.3, 150.69], [37.3, 271.69], [42.7, 274.31]], 9: [[210.06, 229.31], [319.94, 442.69], [326.06, 439.31], [203.94, 232.69]], 10: [[239.33, 222.52], [387.67, 243.48], [388.33, 236.52], [238.67, 229.48]], 11: [[423.87, 146.88], [418.13, 145.12], [375.13, 286.12], [380.87, 287.88]], 12: [[180.13, 52.78], [175.87, 47.22], [0.87, 181.22], [5.13, 186.78]], 13: [[389.11, 230.51], [164.11, 181.51], [163.89, 182.49], [388.89, 231.49]], 14: [[15.86, 316.5], [46.86, 263.5], [4

Для определения, лежит ли точка P внутри прямоугольника ABCD, сложим площади треугольников APB, BPC, CPD, APD. Если их сумма равна площади прямоугольника, то точка принадлежит ему, если больше - то не принадлежит. Площадь выпуклого n-угольника при известных координатах вершин определяем как abs((x1y2-y1x2)+(x2y3-y3x2)...+(xny1-ynx1))/2.

In [26]:
def isPinRectangle(r, P):
    areaRectangle = (abs(r[0][0]*r[1][1]-r[0][1]*r[1][0]+r[1][0]*r[2][1]-r[2][0]*r[1][1]+r[2][0]*r[3][1]-r[3][0]*r[2][1]+r[3][0]*r[0][1]-r[0][0]*r[3][1]))/2
    ABP = abs(r[0][0]*r[1][1]-r[0][1]*r[1][0]+r[1][0]*P[1]-P[0]*r[1][1]+P[0]*r[0][1]-r[0][0]*P[1])/2
    BCP = abs(r[1][0]*r[2][1]-r[1][1]*r[2][0]+r[2][0]*P[1]-P[0]*r[2][1]+P[0]*r[1][1]-r[1][0]*P[1])/2
    CDP = abs(r[2][0]*r[3][1]-r[2][1]*r[3][0]+r[3][0]*P[1]-P[0]*r[3][1]+P[0]*r[2][1]-r[2][0]*P[1])/2
    DAP = abs(r[3][0]*r[0][1]-r[3][1]*r[0][0]+r[0][0]*P[1]-P[0]*r[0][1]+P[0]*r[3][1]-r[3][0]*P[1])/2
    return areaRectangle == ABP + BCP + CDP + DAP

Оставляем вероятность простого касания прямоугольников: 

Взглянем на иллюстрацию https://codechef_shared.s3.amazonaws.com/download/Rectangle%20Intersection.png . Она наглядно демонстрирует возможные случаи пересечения прямоугольников. Видно, что, как правило, действует следующий принцип: если хотя бы одна вершина прямоугольника лежит внутри другого прямоугольника, прямоугольники пересекаются.
Этот принцип нарушается последним случаем (крест). Однако здесь заметны четыре (в отличие от предыдущих) пересечения разных сторон друг между другом.
Используем функцию для определения, пересекаются ли отрезки.

In [27]:
def ccw(A,B,C):
    return (C[1]-A[1])*(B[0]-A[0]) > (B[1]-A[1])*(C[0]-A[0])
def intersect(A,B,C,D):
        return ccw(A,C,D) != ccw(B,C,D) and ccw(A,B,C) != ccw(A,B,D)

Также нужно учесть, что по алгоритму определения нахождения точек, он будет также идентифицировать те, которые находятся на сторонах. Таким образом, ещё нужно учитывать возможные касания: если одна или две точки, единственные, удовлетворяющие площадному условию, лежат на одной стороне другого прямоугольника, то прямоугольники не пересекаются.

In [28]:
def distance(a,b):
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

def is_between(a,c,b):
    return math.isclose((distance(a,c) + distance(c,b)),distance(a,b))

In [88]:
def checkintersect(a,b):
     #стороны прямоугольников. так, потому что уже по часовой стрелке
#    is_between(firstlines[i][0],firstlines[i][1],secondlines[i][0],secondlines[i][1]):
    
    does = False #флаг пересечения
               
    count1 = 0
    for i in a:
        if isPinRectangle(b,i): #нахождение прямоугольника внутри другого целиком
            count1 += 1
    if count1 == 4:
        does = True
    
    count2 = 0
    for i in b:
        if isPinRectangle(a,i): #аналогично наоборот
            count2 += 1
    if count2 == 4:
        does = True
        
    # более сложные случаи
    count = 0
    flags = 0
    flagAB = False
    flagBC = False
    flagCD = False #флаги касания соответствующих сторон
    flagAD = False
    for i in b:
        if isPinRectangle(a,i):
            count += 1
            if not (is_between(a[0],i,a[1]) or is_between(a[1],i,a[2]) or is_between(a[2],i,a[3]) or is_between(a[3],i,a[0])):
                does = True
                break
            elif is_between(a[0],i,a[1]) and flagAB == False: #две точки касания на одной стороне
                flagAB = True
                flags += 1
            elif is_between(a[0],i,a[1]) and flagAB == True:
                break
            elif is_between(a[1],i,a[2]) and flagBC == False:
                flagBC = True
                flags += 1
            elif is_between(a[1],i,a[2]) and flagBC == True:
                break
            elif is_between(a[2],i,a[3]) and flagCD == False:
                flagCD = True
                flags += 1
            elif is_between(a[2],i,a[3]) and flagCD == True:
                break
            elif is_between(a[3],i,a[0]) and flagAD == False:
                flagAD = True
                flags += 1
            elif is_between(a[3],i,a[0]) and flagAD == True:
                break
    if count == flags and count > 1: # Если точки касания принадлежат разным сторонам и их больше одной:
        does = True
             
        
    firstlines = [[a[0],a[1]],[a[1],a[2]],[a[2],a[3]],[a[0],a[3]]] #проверка на крест
    secondlines = [[b[0],b[1]],[b[1],b[2]],[b[2],b[3]],[b[0],b[3]]]
    inter = 0
    for i in range(4):
        for j in range(4):
            if intersect(firstlines[i][0],firstlines[i][1],secondlines[j][0],secondlines[j][1]):
                inter += 1
    if inter == 4:
        does = True
    return does

In [89]:
#Проверим возможные случаи, отрицательные и обозначенные в вышепродемонстрированном чертеже. Пересечение углом:
a = [[1,1],[1,3],[3,3],[3,1]]
b = [[2,2],[2,4],[5,4],[5,2]]

print(checkintersect(a, b))

True


In [90]:
#Пересечение с совпадающими частями сторон (не на схеме)
a = [[1,1],[1,3],[3,3],[3,1]]
b = [[2,1],[2,3],[5,3],[5,1]]

print(checkintersect(a, b))

True


In [91]:
#Пересечение частью
a = [[1,1],[1,4],[5,4],[5,1]]
b = [[3,2],[3,3],[7,3],[7,2]]

print(checkintersect(a, b))

True


In [92]:
#Полное нахождение внутри
a = [[1,1],[1,4],[6,4],[6,1]]
b = [[2,2],[2,3],[5,3],[5,2]]

print(checkintersect(a, b))

True


In [93]:
#Полное нахождение внутри при принадлежности сторон
a = [[1,1],[1,3],[4,3],[4,1]]
b = [[2,1],[2,3],[3,3],[3,1]]

print(checkintersect(a, b))

True


In [94]:
#Касание одной точкой
a = [[1,1],[1,4],[5,4],[5,1]]
b = [[5,4],[5,6],[6,6],[6,4]]

print(checkintersect(a, b))

False


In [95]:
#Совпадающая часть стороны

a = [[1,1],[1,4],[5,4],[5,1]]
b = [[5,2],[5,3],[7,3],[7,2]]

print(checkintersect(a, b))

False


In [96]:
#Крестообразное пересечение

a = [[2,1],[2,5],[4,5],[4,1]]
b = [[1,2],[1,3],[5,3],[5,2]]

print(checkintersect(a, b))

True


Вернёмся к нашей работе. Определим все пары пересекающихся прямоугольников

In [42]:
from itertools import combinations #Все возможные попарные сочетания прямоугольников по индексам
    
numslist = [i+1 for i in range(num)]
print(numslist)
pairs = [list(i) for i in combinations(numslist,2)]
print(pairs)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
[[1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 7], [1, 8], [1, 9], [1, 10], [1, 11], [1, 12], [1, 13], [1, 14], [1, 15], [2, 3], [2, 4], [2, 5], [2, 6], [2, 7], [2, 8], [2, 9], [2, 10], [2, 11], [2, 12], [2, 13], [2, 14], [2, 15], [3, 4], [3, 5], [3, 6], [3, 7], [3, 8], [3, 9], [3, 10], [3, 11], [3, 12], [3, 13], [3, 14], [3, 15], [4, 5], [4, 6], [4, 7], [4, 8], [4, 9], [4, 10], [4, 11], [4, 12], [4, 13], [4, 14], [4, 15], [5, 6], [5, 7], [5, 8], [5, 9], [5, 10], [5, 11], [5, 12], [5, 13], [5, 14], [5, 15], [6, 7], [6, 8], [6, 9], [6, 10], [6, 11], [6, 12], [6, 13], [6, 14], [6, 15], [7, 8], [7, 9], [7, 10], [7, 11], [7, 12], [7, 13], [7, 14], [7, 15], [8, 9], [8, 10], [8, 11], [8, 12], [8, 13], [8, 14], [8, 15], [9, 10], [9, 11], [9, 12], [9, 13], [9, 14], [9, 15], [10, 11], [10, 12], [10, 13], [10, 14], [10, 15], [11, 12], [11, 13], [11, 14], [11, 15], [12, 13], [12, 14], [12, 15], [13, 14], [13, 15], [14, 15]]


Пересекающиеся пары наших прямоугольников

In [67]:
interlist = []
for i in pairs:
    if checkintersect(rectdict[i[0]],rectdict[i[1]]):
        interlist.append(i)
print(interlist)

[[1, 5], [1, 9], [1, 10], [1, 13], [1, 15], [2, 4], [3, 13], [4, 14], [5, 10], [5, 13], [5, 15], [7, 15], [9, 15]]


In [61]:
interlist = sorted([sorted(x) for x in interlist])
# по идее эта операция не нужна, так как на предыдущем шаге всё уже 
# выстроено в нужном порядке, но производим его для очистки совести

Объединим списки с общими элементами:

In [69]:
from collections import defaultdict

def connected_components(lists):
    neighbors = defaultdict(set)
    seen = set()
    for each in lists:
        for item in each:
            neighbors[item].update(each)
    def component(node, neighbors=neighbors, seen=seen, see=seen.add):
        nodes = set([node])
        next_node = nodes.pop
        while nodes:
            node = next_node()
            see(node)
            nodes |= neighbors[node] - seen
            yield node
    for node in neighbors:
        if node not in seen:
            yield sorted(component(node))

grouped = list(connected_components(interlist))
print(grouped)

[[1, 3, 5, 7, 9, 10, 13, 15], [2, 4, 14]]


Спустя энное число попыток наконец-то сгенерировалось пятнадцать прямоугольников, разбившихся на РАЗНЫЕ группы по пересечению. Выведем данные группы построчно.

In [74]:
number = 1
for i in grouped:
    print('Пересекающаяся группа номер '+str(number)+': прямоугольники номер '+', '.join([str(m) for m in i])+';')
    number += 1
from itertools import chain
unnested = list(chain.from_iterable(grouped))
if unnested == numslist:
    print('Ни с чем не пересекающиеся прямоугольники отсутствуют.')
else:
    solitary = [i for i in numslist if i not in unnested]
    print('Ни с чем не пересекающиеся прямоугольники: '+', '.join([str(d) for d in solitary])+'.')

Пересекающаяся группа номер 1: прямоугольники номер 1, 3, 5, 7, 9, 10, 13, 15;
Пересекающаяся группа номер 2: прямоугольники номер 2, 4, 14;
Ни с чем не пересекающиеся прямоугольники: 6, 8, 11, 12.
